In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
from sklearn.linear_model import LogisticRegression

## Part 1: Off-Policy Evaluation: Warm-up

The following code generates logged data from a "tabular" policy. 

In [1]:
def logging_data_tabular(n, num_contexts, num_actions, seed=1):
    '''
    tabular: Uniform distribution on contexts. Distribution on actions chosen randomly from the simplex on actions.
    The random seed is set to 10 to ensure the parameters are similar each round. It is then set to seed to ensure randomization
    before rounds
    
    Returns
    - df: dataframe with n rows and columns: context, action, propensity, reward  
    - r: num_contexts x num_actions: reward matrix
    - pi0: num_contexts x num_actions: matrix where pi0[i,j] = pi0(a_j|x_i)
    '''
    np.random.seed(10)
    r = np.random.rand(num_contexts, num_actions)
    pi0 = np.random.rand(num_contexts, num_actions)
    pi0 = pi0/pi0.sum(axis=1)[:,np.newaxis]
    
    np.random.seed(seed)
    contexts = list(range(num_contexts))
    actions = list(range(num_actions))
    data = []
    for i in range(n):
        ct = contexts[np.random.choice(contexts)]
        at = actions[np.random.choice(actions, p=pi0[ct])]
        pt = pi0[ct,at]
        rt = r[ct,at] + .1*np.random.randn()
        data.append((ct,at,pt,rt, ))
    df = pd.DataFrame(data, columns=['context', 'action', 'propensity', 'reward'])
    df = df.astype(dtype={"context":"int64", "action":"int64"})
    return df, r, pi0
    

I've also provided the following ``UniformPolicy`` class which samples each action with the same uniform probability

In [2]:
class UniformPolicy():
    def __init__(self, num_contexts, num_actions):
        self.num_contexts = num_contexts
        self.num_actions = num_actions
    
    def pi_x(self, ctx):
        '''
        Return pi(-|x) which is a vector in R^|A|
        '''
        return np.ones(self.num_actions)/self.num_actions
    
    def pi_a_x(self, ctx, a):
        '''
        Return pi(a|x). 
        In the case of a deterministic policy should be a binary variable.
        '''
        return 1/self.num_actions
    
    def V(self, r):
        '''
        Return the true value of the task given the true rewards and 
        Assume context distribution is uniform.
        '''
        v = 0
        for ctx in range(self.num_contexts):
            for action in range(self.num_actions):
                v += 1/self.num_contexts*self.pi_a_x(ctx, action)*r[ctx, action]
        return v

The following example shows an example of pulling a dataset and computing $V(\pi_{unif})$ on the true reward function returned from the logging method above. Please run this code and understand the previous code fully before continuing.

In [ ]:
d,r,pi0 = logging_data_tabular(10, 50, 2)
uniform_policy = UniformPolicy(50,2)
v_star = uniform_policy.V(r)

Please implement the IPS, direct method, and doubly robust estimators. You may assume that the policy object has the same methods as a ``UniformPolicy`` class above. I've given you the ``_rhat`` method which computes an empirical reward matrix given the data.

In [3]:
def ips(policy, data, clip=0):
    ## TODO implement IPS
    return vhat_ips

def _rhat(data, num_contexts, num_actions):
    rhat = np.zeros((num_contexts, num_actions))
    for ctx in range(num_contexts):
        for action in range(num_actions):
            dd = data.query(f'context=={ctx} and action=={action}')
            rhat[ctx,action] = dd['reward'].sum()/max(len(dd), 1)
    return rhat
    
def dm(policy, data):
    ## TODO implement the direct method
    ## You may want to use the _rhat method above
    return vhat_dm

def dr(policy, data, clip=0):
   ## TODO implement the doubly robust estimator
    return vhat_dr

Now run the simulation described in problem 2 of part 1 of the homework. I've tried to give you some boilerplate code. Feel free to modify anything below.

In [ ]:
reps = 20
ns = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
runs_ips = np.zeros((len(ns), reps))
runs_dm = np.zeros((len(ns), reps))
runs_dr = np.zeros((len(ns), reps))


d,r,pi0 = logging_data_tabular(10, 50, 2)
uniform_policy = UniformPolicy(50,2)
v_star = uniform_policy.V(r)

for j, n in enumerate(ns):    
    for i in range(reps):
        '''
        TODO:
        a) Draw a dataset of size n from logging_data_tabular
        b) Compute vhat(pi_unif) using IPS, DM, DR estimators
        c) Compute the mean squared error for each estimator relative to v_star
        d) Store the resulting answer in runs_ips[j,i], runs_dm[j,i], runs_dr[j,i] respectively.
        ''' 
        

#plt.axhline(v_star, c='blue')
plt.plot(ns, runs_ips.mean(axis=1), label='V_IPS', c='blue')
plt.errorbar(ns, runs_ips.mean(axis=1), yerr=runs_ips.std(axis=1), c='blue')
plt.plot(ns, runs_dm.mean(axis=1), label='V_dm',c='green')
plt.errorbar(ns, runs_dm.mean(axis=1), yerr=runs_dm.std(axis=1),c='green')
plt.plot(ns, runs_dr.mean(axis=1), label='V_dr',c='orange')
plt.errorbar(ns, runs_dr.mean(axis=1), yerr=runs_dr.std(axis=1),c='orange')
plt.yscale('log')
plt.legend()
plt.show()


## Part 2: Off-Policy Evaluation: Learning

The following code generates logged data from a "softmax" policy that is designed to be close to the true optimal deterministic policy.

In [5]:
def logging_data_logistic(n, num_contexts, num_actions, dim=2, seed=1, eta=.1):
    '''
    logistic: Please see homework for description.
    '''
    np.random.seed(10)
    r = np.zeros((num_contexts, num_actions))
    contexts = list(range(num_contexts))
    actions = list(range(num_actions))
    contexts_X = np.random.randn(num_contexts, dim)
    contexts_A = np.random.randn(num_contexts, dim)
    theta = np.random.rand(dim,dim)
    for i in range(num_contexts):
        for j in range(num_actions):
            r[i,j] = max(contexts_X[i].T@theta@contexts_A[j],0)
    pi0 = np.exp(eta*r+ .05*np.random.randn())
    pi0 = pi0/pi0.sum(axis=1)[:,np.newaxis] 
    # policy - sample theta and look at probability given inner products of xa^top with theta 
    
    np.random.seed(seed)
    data = []
    for i in range(n):
        ct = contexts[np.random.choice(contexts)]
        at = actions[np.random.choice(actions, p=pi0[ct])]
        pt = pi0[ct,at]
        rt = max(r[ct,at] + .1*np.random.randn(), 0)
        data.append((ct,at,pt,rt))
    data.append((ct,at,pt,rt))
    df = pd.DataFrame(data, columns=['context', 'action', 'propensity', 'reward'])
    df = df.astype(dtype={"context":"int64", "action":"int64"})
    return df, r, pi0, contexts_X, contexts_A

I've also provided the ``LogisticDeterministicPolicy`` class which provides a wrapper class to convert a multi-class classifier into a policy

In [6]:
class LogisticDeterministicPolicy():
    def __init__(self, num_contexts, num_actions, model, contexts_X, contexts_A):
        self.num_contexts = num_contexts
        self.num_actions = num_actions
        self.model = model
    
    def pi_x(self, ctx):
        '''
        Return pi(-|x) which is a vector in R^|A|
        '''
        p = np.zeros(num_actions)
        a = model.predict(contexts_X[ctx][np.newaxis,:])[0]
        p[a] = 1
        return p
    
    def pi_a_x(self, ctx, a):
        '''
        Return pi(a|x). 
        In the case of a deterministic policy should be a binary variable.
        '''
        a_predict = model.predict(contexts_X[int(ctx)][np.newaxis,:])[0]
        return a_predict == a
    
    def V(self, r):
        '''
        Return the true value of the task given the true rewards and 
        Assume context distribution is uniform.
        '''
        v = 0
        for ctx in range(self.num_contexts):
            for action in range(self.num_actions):
                v += 1/self.num_contexts*self.pi_a_x(ctx, action)*r[ctx, action]
        return v

Now the main exercise. Using the reduction to multi-class classification, implement ``ipw_learner`` below. Use the ``LogisticRegression`` class from ``sklearn`` as your underlying training procedure. I've provided some testing code in the cell below. 

In [7]:
def ipw_learner(data, contexts_X, contexts_A):
    # TODO
    # return a LogisticDeterministicPolicy object
    return policy

In [ ]:
## Testing code to check your implementation of ipw_learner.
d, r, pi0, contexts_X, contexts_A = logging_data_logistic(1000, 50, 2, seed=50)
learned_policy = ipw_learner(d, contexts_X, contexts_A)
v_star = logistic_policy.V(r)
print('True value of learned policy', v_star)
# Value of true best deterministic policy
m = 0
for i in range(50):
    m += np.max(r[i])
print('best deterministic policy', m/50)

Finallly run a simulation as described in the text. I have tried to give you some warmup code.

In [ ]:
reps = 20
ns = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
runs_ips = np.zeros((len(ns), reps))
runs_dm = np.zeros((len(ns), reps))
runs_dr = np.zeros((len(ns), reps))


d, r, pi0, contexts_X, contexts_A = logging_data_logistic(1000, 50, 2, seed=50)

for j, n in enumerate(ns):    
    for i in range(reps):
        '''
        TODO:
        a) Draw a dataset of size n from logging_data_tabular
        b) Learn a pi_hat from ipw_learner
        c) Compute vhat(pi_hat) using IPS, DM, DR estimators
        d) Compute the mean squared error for each estimator relative to v_star
        e) Store the resulting answer in runs_ips[j,i], runs_dm[j,i], runs_dr[j,i] respectively.
        ''' 

plt.plot(ns, runs_ips.mean(axis=1), label='V_IPS', c='blue')
plt.errorbar(ns, runs_ips.mean(axis=1), yerr=runs_ips.std(axis=1), c='blue')
plt.plot(ns, runs_dm.mean(axis=1), label='V_dm',c='green')
plt.errorbar(ns, runs_dm.mean(axis=1), yerr=runs_dm.std(axis=1),c='green')
plt.plot(ns, runs_dr.mean(axis=1), label='V_dm',c='orange')
plt.errorbar(ns, runs_dr.mean(axis=1), yerr=runs_dr.std(axis=1),c='orange')
plt.legend()
plt.show()